In [14]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader
from datasets import load_dataset

from haystack.utils import print_answers

from haystack.nodes import ElasticsearchRetriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import DocumentSearchPipeline, ExtractiveQAPipeline

In [15]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [22]:
import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT  # as daemon
)
# wait until ES has started
# ! sleep 30

In [23]:
# Connect to Elasticsearch

from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="0.0.0.0", username="", password="", index="document")

/home/hadoop/miniconda3/envs/torch/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [25]:
from datasets import load_from_disk

train= load_from_disk('./data/train_re')
valid= load_from_disk('./data/valid_re')
test= load_from_disk('./data/test_re')

In [26]:
from tqdm import tqdm
retrieve_dataset= []

for c in tqdm(train):
    retrieve_dataset.append(dict(content= c['context'], meta= {'question': c['question'], 'output': c['output']}))

for c in tqdm(test):
    retrieve_dataset.append(dict(content= c['context'], meta= {'question': c['question'], 'output': c['output']}))

for c in tqdm(valid):
    retrieve_dataset.append(dict(content= c['context'], meta= {'question': c['question'], 'output': c['output']})) 

100%|██████████| 272/272 [00:00<00:00, 5627.38it/s]


In [27]:
document_store.write_documents(retrieve_dataset)

/home/hadoop/miniconda3/envs/torch/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [28]:
from haystack.nodes import ElasticsearchRetriever, FARMReader

retriever = ElasticsearchRetriever(document_store=document_store)

In [29]:
from haystack.pipelines import DocumentSearchPipeline, ExtractiveQAPipeline

pipe = DocumentSearchPipeline(retriever)

In [35]:
k= 50
cnt= 0

for i, t in tqdm(enumerate(test)):
    question, labels= t['question'], t['context']
    topk= pipe.run(query= question, params={"Retriever": {"top_k": k}})

    topk_passage= []

    for j in range(k):
        topk_passage.append(topk['documents'][j].content)

    if labels in topk_passage:
        # print(labels)
        cnt+=1

print(f'test retriever accuracy : {cnt/len(test)*100}%')


281it [00:13, 21.27it/s]

test retriever accuracy : 75.80071174377224%
